In [1]:
import findspark
findspark.init()

In [2]:
import pandas as pd
from pyspark.sql import SparkSession

load by pandas

In [3]:
df = pd.read_csv('/home/eric/Sync/datasets/moviedata/moviedata-lite/ratings.csv')
df.head()

,RATING_ID,USER_MD5,MOVIE_ID,RATING,RATING_TIME
0,1359352573,0ab7e3efacd56983f16503572d2b9915,5113101,2,2018-09-05 19:42:07
1,1598245094,84dfd3f91dd85ea105bc74a4f0d7a067,5113101,1,2019-07-09 14:52:07
2,311937819,c9a47fd59b55967ceac07cac6d5f270c,3718526,3,2010-11-05 22:15:44
3,457663846,18cbf971bdf17336056674bb8fad7ea2,3718526,4,2011-11-14 22:31:02
4,313277849,47e69de0d68e6a4db159bc29301caece,3718526,4,2010-11-09 12:41:11


write to parquet

In [4]:
df.to_parquet('/tmp/ratings.parquet')

In [5]:
!ls /tmp/ratings*

/tmp/ratings.parquet


read back by pandas

In [6]:
pd.read_parquet('/tmp/ratings.parquet').head()

,RATING_ID,USER_MD5,MOVIE_ID,RATING,RATING_TIME
0,1359352573,0ab7e3efacd56983f16503572d2b9915,5113101,2,2018-09-05 19:42:07
1,1598245094,84dfd3f91dd85ea105bc74a4f0d7a067,5113101,1,2019-07-09 14:52:07
2,311937819,c9a47fd59b55967ceac07cac6d5f270c,3718526,3,2010-11-05 22:15:44
3,457663846,18cbf971bdf17336056674bb8fad7ea2,3718526,4,2011-11-14 22:31:02
4,313277849,47e69de0d68e6a4db159bc29301caece,3718526,4,2010-11-09 12:41:11


PySpark

In [7]:
spark = (SparkSession
    .builder
    .appName("pyspark-pandas-koalas")
    .getOrCreate())

In [8]:
df = spark.read.csv('/home/eric/Sync/datasets/moviedata/moviedata-lite/ratings.csv', header=True)
df.repartition(1).write.mode('overwrite').parquet('/tmp/pyspark_ratings')

In [9]:
df.show()

+----------+--------------------+--------+------+-------------------+
| RATING_ID|            USER_MD5|MOVIE_ID|RATING|        RATING_TIME|
+----------+--------------------+--------+------+-------------------+
|1359352573|0ab7e3efacd56983f...| 5113101|     2|2018-09-05 19:42:07|
|1598245094|84dfd3f91dd85ea10...| 5113101|     1|2019-07-09 14:52:07|
| 311937819|c9a47fd59b55967ce...| 3718526|     3|2010-11-05 22:15:44|
| 457663846|18cbf971bdf173360...| 3718526|     4|2011-11-14 22:31:02|
| 313277849|47e69de0d68e6a4db...| 3718526|     4|2010-11-09 12:41:11|
| 798211658|86dd268419a025131...|10514820|     3|2014-04-19 01:30:48|
| 582827065|93ce30280bf1b1160...|10514820|     3|2012-09-22 12:52:36|
| 247354289|946c40119cf6ac3f9...| 4824996|     4|2010-05-04 13:02:36|
| 560603476|c98155000a3420758...| 4824996|     5|2012-07-25 22:03:32|
| 345508641|bd3ba3fc3a4f3c925...| 4824996|     5|2011-01-26 17:18:50|
| 919041772|e4e41a7765b5fe37b...|10514820|     2|2015-05-04 17:23:24|
| 665120053|7a48f067

In [10]:
!ls /tmp/pyspark_ratings -al

total 72
drwxrwxr-x  2 eric eric  4096 May  7 11:24 .
drwxrwxrwt 31 root root  4096 May  7 11:24 ..
-rw-r--r--  1 eric eric 55326 May  7 11:24 part-00000-f173425e-6b50-4327-8907-b8aed0b3863a-c000.snappy.parquet
-rw-r--r--  1 eric eric   444 May  7 11:24 .part-00000-f173425e-6b50-4327-8907-b8aed0b3863a-c000.snappy.parquet.crc
-rw-r--r--  1 eric eric     0 May  7 11:24 _SUCCESS
-rw-r--r--  1 eric eric     8 May  7 11:24 ._SUCCESS.crc


read back

In [11]:
df = spark.read.parquet('/tmp/pyspark_ratings')
df.show(3)

+----------+--------------------+--------+------+-------------------+
| RATING_ID|            USER_MD5|MOVIE_ID|RATING|        RATING_TIME|
+----------+--------------------+--------+------+-------------------+
|1359352573|0ab7e3efacd56983f...| 5113101|     2|2018-09-05 19:42:07|
|1598245094|84dfd3f91dd85ea10...| 5113101|     1|2019-07-09 14:52:07|
| 311937819|c9a47fd59b55967ce...| 3718526|     3|2010-11-05 22:15:44|
+----------+--------------------+--------+------+-------------------+
only showing top 3 rows



In [12]:
pd.read_parquet('/tmp/pyspark_ratings').head(3)

,RATING_ID,USER_MD5,MOVIE_ID,RATING,RATING_TIME
0,1359352573,0ab7e3efacd56983f16503572d2b9915,5113101,2,2018-09-05 19:42:07
1,1598245094,84dfd3f91dd85ea105bc74a4f0d7a067,5113101,1,2019-07-09 14:52:07
2,311937819,c9a47fd59b55967ceac07cac6d5f270c,3718526,3,2010-11-05 22:15:44


koalas

In [13]:
import databricks.koalas as ks

In [14]:
df = ks.read_csv('/home/eric/Sync/datasets/moviedata/moviedata-lite/ratings.csv')
df.head(3)

,RATING_ID,USER_MD5,MOVIE_ID,RATING,RATING_TIME
0,1359352573,0ab7e3efacd56983f16503572d2b9915,5113101,2,2018-09-05 19:42:07
1,1598245094,84dfd3f91dd85ea105bc74a4f0d7a067,5113101,1,2019-07-09 14:52:07
2,311937819,c9a47fd59b55967ceac07cac6d5f270c,3718526,3,2010-11-05 22:15:44


In [15]:
df.to_parquet('/tmp/koalas_ratings')

In [16]:
!ls /tmp/koalas_ratings -al

total 68
drwxrwxr-x  2 eric eric  4096 May  7 11:24 .
drwxrwxrwt 31 root root  4096 May  7 11:24 ..
-rw-r--r--  1 eric eric 50200 May  7 11:24 part-00000-c5133d54-11d1-4fb8-8434-a158ca297867-c000.snappy.parquet
-rw-r--r--  1 eric eric   404 May  7 11:24 .part-00000-c5133d54-11d1-4fb8-8434-a158ca297867-c000.snappy.parquet.crc
-rw-r--r--  1 eric eric     0 May  7 11:24 _SUCCESS
-rw-r--r--  1 eric eric     8 May  7 11:24 ._SUCCESS.crc


In [17]:
df = spark.read.parquet('/tmp/koalas_ratings')
df.show(3)

+----------+--------------------+--------+------+-------------------+
| RATING_ID|            USER_MD5|MOVIE_ID|RATING|        RATING_TIME|
+----------+--------------------+--------+------+-------------------+
|1359352573|0ab7e3efacd56983f...| 5113101|     2|2018-09-05 19:42:07|
|1598245094|84dfd3f91dd85ea10...| 5113101|     1|2019-07-09 14:52:07|
| 311937819|c9a47fd59b55967ce...| 3718526|     3|2010-11-05 22:15:44|
+----------+--------------------+--------+------+-------------------+
only showing top 3 rows



In [18]:
pd.read_parquet('/tmp/koalas_ratings').head(3)

,RATING_ID,USER_MD5,MOVIE_ID,RATING,RATING_TIME
0,1359352573,0ab7e3efacd56983f16503572d2b9915,5113101,2,2018-09-05 11:42:07
1,1598245094,84dfd3f91dd85ea105bc74a4f0d7a067,5113101,1,2019-07-09 06:52:07
2,311937819,c9a47fd59b55967ceac07cac6d5f270c,3718526,3,2010-11-05 14:15:44


In [19]:
spark.stop()